# I. Introduction/Business Problem

This project will be a way of comparing colleges. The user will be able to perform comparison and clustering based on details about the universities themselves (including tuition/fees, acceptance rate, student body size, etc.) as well as attractions in the cities in which they are located.

I will specifically be working with the eight Ivy League schools (Brown University, Columbia University, Cornell University, Dartmouth College, Harvard University, Princeton University, University of Pennsylvania, and Yale University), however the code could be easily modified to compare other universities/cities.

This tool is designed to help students make choices about which university to attend.

# II. Data

For this project, I will be using Foursquare location data to explore the attractions/amenities nearby each of the universities. This would be helpful for students looking to narrow their choices based on the type of city they'd like to live in: Is it important to them to have lots of restaurants nearby? Or museums? Or parks? Or music venues?

For the school-related data, I will be using data from these two URLs: 
http://blog.collegetuitioncompare.com/2015/05/ivy-league-2015-2016-estimated-tuition.html
https://www.collegetuitioncompare.com/best-schools/ivy-league/

From these two sources, I can use webscraping to obtain information about 2020 tuition/fees and on-campus housing costs, as well as the acceptance rates, SAT scores, graduation rate, and student/faculty ratio. All of these pieces of information could be useful to a prospective student trying to choose a university.